In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [814 kB]
Err:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
  File has unexpected size (815853 != 814314). Mirror sync in progress? [IP: 152.195.19.142 443]
  Hashes of expected file:
   - Filesize:814314 [weak]
   - SHA256:257071ac3a46f8e8ba340c2bd6b88466ff26e4cb0c4b6

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [4]:
cleaned_df = df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [7]:
vine_df = cleaned_df.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

In [8]:
helpful_df = vine_df.filter("total_votes>=20")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [15]:
percent_helpful = helpful_df.filter("helpful_votes / total_votes >= .5")
percent_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [19]:
vine_review_yes = helpful_df.filter(helpful_df["vine"] == "Y")
vine_review_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|                N|
| R886ADSI1HBU5|          5|           88|         97|   Y|                N|
|R3TTJTAND1CCSV|          5|           21|         23|   Y|                N|
|R1W39WXYWNFJBG|          5|           32|         35|   Y|     

In [20]:
vine_review_no = helpful_df.filter(helpful_df["vine"] == "N")
vine_review_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [23]:
from pyspark.sql.functions import count

In [50]:
total_no_reviews = vine_df.select(count("vine")).show()

+-----------+
|count(vine)|
+-----------+
|    6908145|
+-----------+



In [51]:
total_helpful_reviews = helpful_df.select(count("vine")).show()

+-----------+
|count(vine)|
+-----------+
|      96147|
+-----------+



In [26]:
no_of_reviews_yes = vine_review_yes.select(count("vine")).show()

+-----------+
|count(vine)|
+-----------+
|       1820|
+-----------+



In [27]:
no_of_reviews_no = vine_review_no.select(count("vine")).show()

+-----------+
|count(vine)|
+-----------+
|      94327|
+-----------+



In [36]:
vine_five_stars = vine_review_yes.filter(vine_review_yes["star_rating"] == "5").select(count("vine")).show()


+-----------+
|count(vine)|
+-----------+
|        788|
+-----------+



In [37]:
non_vine_five_stars = vine_review_no.filter(vine_review_no["star_rating"] == "5").select(count("vine")).show()

+-----------+
|count(vine)|
+-----------+
|      37602|
+-----------+



In [49]:
from pyspark.sql.functions import col,when,count,lit
ratings_total_df = helpful_df.groupBy("vine").agg(
    count(col("vine")).alias("Total Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total Five Star Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("% Five Star Rating")).show()


+----+-------------+-----------------------+------------------+
|vine|Total Reviews|Total Five Star Reviews|% Five Star Rating|
+----+-------------+-----------------------+------------------+
|   Y|         1820|                    788|  43.2967032967033|
|   N|        94327|                  37602|39.863453730109086|
+----+-------------+-----------------------+------------------+

